In [1]:
# load dataframe
import pandas as pd
pd.options.display.max_columns = 999
df = pd.read_csv('Droughts_satelite_data_events_1month_normalized_corrected.csv', index_col=0)
df = df.drop(columns=['month', 'day', 'year'])
df['date'] = pd.to_datetime(df['date'])
df.head()

,Country,District,date,NDVI,EVI,precipitation_per_hour_v1,precipitation_per_hour_v2,surface_temperature_daytime,surface_temperature_nighttime,evapotranspiration,rainfall,SoilMoisture00_10cm,SoilMoisture10_40cm,SoilMoisture40_100cm,SoilMoisture100_200cm,SoilTemperature00_10cm,SoilTemperature10_40cm,SoilTemperature40_100cm,SoilTemperature100_200cm,air_temperature,wind_speed,SPEI_1month,SPEI_2month,SPEI_3month,SPEI_4month,SPEI_5month,SPEI_6month,SPEI_7month,SPEI_8month,SPEI_9month,SPEI_10month,SPEI_11month,SPEI_12month,drought_reported,drought_news_article,drought_desinventar,drought_estimated
0,Uganda,ABIM,2000-03-01,-1.874641,-1.570809,-1.217134,-1.210825,1.914201,1.651377,-1.632338,-1.362386,-1.683524,-1.594857,-1.053627,-1.268783,2.063513,2.163876,2.106542,1.816289,2.028033,0.979077,-0.776332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False
1,Uganda,ABIM,2000-04-01,-1.206775,-1.143460,0.554526,0.337322,1.068590,0.768848,0.343810,0.380445,-0.248744,-1.965811,-1.469958,-1.413054,0.806640,1.261010,1.648963,1.964269,1.483038,0.049384,-0.231381,-0.569433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False
2,Uganda,ABIM,2000-05-01,0.544968,0.484206,0.726150,0.654021,-0.209722,0.109642,1.241335,1.416795,0.717814,0.108791,-0.620995,-0.999464,-0.426027,0.004497,0.442751,1.120464,0.312072,-0.720565,0.874819,0.344453,0.075238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False
3,Uganda,ABIM,2000-06-01,0.853214,0.948032,0.974735,0.485003,-0.526822,-0.525072,0.585442,1.246105,0.707451,-0.143555,-0.602664,-0.723248,-0.853696,-0.416556,-0.098606,0.309552,-0.555277,-1.249491,1.353014,1.302469,0.917779,0.711107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False
4,Uganda,ABIM,2000-07-01,1.180418,1.321811,0.368088,-0.002803,-1.099682,-1.324882,0.660043,1.021191,1.218931,1.271444,0.786433,0.311089,-1.531640,-1.284248,-1.029472,-0.551573,-1.411498,-1.310899,0.223291,0.865742,1.070958,0.836407,0.635385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False


In [2]:
len(df[df['drought_estimated']])

305

In [3]:
# define target and predictors
y_label = ['drought_estimated']
X_labels = ['NDVI', 'EVI',
       'precipitation_per_hour_v1', 'precipitation_per_hour_v2',
       'surface_temperature_daytime', 'surface_temperature_nighttime',
       'evapotranspiration', 'rainfall', 'SoilMoisture00_10cm',
       'SoilMoisture10_40cm', 'SoilMoisture40_100cm', 'SoilMoisture100_200cm',
       'SoilTemperature00_10cm', 'SoilTemperature10_40cm',
       'SoilTemperature40_100cm', 'SoilTemperature100_200cm',
       'air_temperature', 'wind_speed', 'SPEI_1month', 'SPEI_2month', 'SPEI_3month', 'SPEI_4month',
       'SPEI_5month', 'SPEI_6month', 'SPEI_7month', 'SPEI_8month',
       'SPEI_9month', 'SPEI_10month', 'SPEI_11month', 'SPEI_12month']

In [4]:
# --- define utility functions ---
from statistics import median
import numpy as np

def crop_window_around_drought(df, drought_label, lower_bound=12, upper_bound=12):
    """ slice dataframe in time windows around certain events
    window size is [event_index-lower_bound, event_index+upper_bound]
    """
    idx = df.index.get_indexer_for(df[df[drought_label]].index)
    if len(idx) > 0:
        df_cropped = df.iloc[np.unique(np.concatenate([np.arange(max(i-lower_bound,0),
                                                                 min(i+upper_bound+1, len(df))) for i in idx]))]
    else:
        df_cropped = pd.DataFrame(columns=df.columns)
    return df_cropped

In [5]:
# crop around events
df_cropped = pd.DataFrame(columns=df.columns)

for dist in df.District.unique():
    
    # select district
    group = df[df['District']==dist]   
    
    # crop around event
    group = crop_window_around_drought(group, y_label[0])
    
    df_cropped = pd.concat([df_cropped, group], ignore_index=True)

print(len(df), '-->', len(df_cropped))
df = df_cropped.copy()

39532 --> 5972


In [8]:
import dateutil.relativedelta

n_time_steps = 6

X_labels_time = []
labels_all = y_label.copy()
X_labels_all = []
for ts in range(0, n_time_steps+1):
    add = [x+'_'+str(ts) for x in X_labels]
    X_labels_time.append(add)
    labels_all.extend(add)
    X_labels_all.extend(add)

# prepare final dataframe (dates, district)
districts = df['District'].unique()
months = pd.date_range(start='1/1/2000', end=df.index.min(), freq='MS') 
df_final = pd.DataFrame(index=pd.MultiIndex.from_product([districts, months],
                                                         names=['district', 'date']),
                       columns = labels_all)
# df_final.head()

for ix, row in df.iterrows():
    date_start = pd.to_datetime(row['date'])
    for t in range(0, n_time_steps+1):
        date_old = date_start - dateutil.relativedelta.relativedelta(months=t)
        try:
            data_old = df[(df.date==date_old) & (df.District==row['District'])]
            for col_num, col_name in enumerate(X_labels_time[t]):
                df_final.at[(row['District'], date_start), col_name] = data_old[X_labels[col_num]].values[0]
        except:
            continue
    df_final.at[(row['District'], date_start), 'drought_estimated'] = row['drought_estimated']

df_final.drought_estimated = df_final.drought_estimated.astype(bool)
df_final.head()

drought_estimated    NDVI_0     EVI_0  \
district date                                                
ABIM     2001-02-01              False  -1.74053  -1.57638   
         2001-03-01              False  -1.71381  -1.38037   
         2001-04-01              False  0.309868  0.205156   
         2001-05-01              False   1.09935   1.16614   
         2001-06-01              False  0.439057   0.83289   

                    precipitation_per_hour_v1_0 precipitation_per_hour_v2_0  \
district date                                                                 
ABIM     2001-02-01                    -1.40918                    -1.28021   
         2001-03-01                    0.956518                     1.51377   
         2001-04-01                     1.01533                    0.610817   
         2001-05-01                    0.726755                    0.492181   
         2001-06-01                    0.400655                    0.242266   

                    surface_temperature_daytime_0  \
district date                                       
ABIM     2001-02-01                       1.79353   
         2001-03-01                       1.84005   
         2001-04-01                      0.195604   
         2001-05-01                     -0.702476   
         2001-06-01                      -1.10912   

                    surface_temperature_nighttime_0 evapotranspiration_0  \
district date                                                              
ABIM     2001-02-01                         1.39024             -1.68457   
         2001-03-01                         1.53435            -0.323192   
         2001-04-01                        0.458223             0.857911   
         2001-05-01                       -0.124684              1.22737   
         2001-06-01                       -0.664326             0.707593   

                    rainfall_0 SoilMoisture00_10cm_0 SoilMoisture10_40cm_0  \
district date                                                                
ABIM     2001-02-01   -1.36407              -1.54566             -0.630702   
         2001-03-01  0.0373043             -0.489599             -0.468632   
         2001-04-01   0.700849              0.330451             -0.114672   
         2001-05-01   0.930721              0.791186              0.343755   
         2001-06-01   0.831024              0.987119              0.711882   

                    SoilMoisture40_100cm_0 SoilMoisture100_200cm_0  \
district date                                                        
ABIM     2001-02-01             -0.0802244               -0.529815   
         2001-03-01              -0.245643               -0.660479   
         2001-04-01              -0.391943                -0.78128   
         2001-05-01             -0.0756078               -0.578316   
         2001-06-01               0.341294               -0.145116   

                    SoilTemperature00_10cm_0 SoilTemperature10_40cm_0  \
district date                                                           
ABIM     2001-02-01                  1.85097                  1.77436   
         2001-03-01                  1.05294                  1.35844   
         2001-04-01                 0.244515                 0.504204   
         2001-05-01                -0.462675                -0.167721   
         2001-06-01                 -1.35222                -0.956833   

                    SoilTemperature40_100cm_0 SoilTemperature100_200cm_0  \
district date                                                              
ABIM     2001-02-01                   1.52761                    1.18222   
         2001-03-01                   1.60155                    1.66377   
         2001-04-01                  0.835792                     1.2598   
         2001-05-01                  0.132473                   0.594789   
         2001-06-01                 -0.645847                  -0.172194   

                    air_temperature_0 wind_s

In [9]:
df_final.to_csv('df_analysis.csv')

In [51]:
# impose class balance, get training indices
pos_samples = df_final[df_final.drought_estimated]
num_samples = len(pos_samples)
neg_samples = df_final[~df_final.drought_estimated].sample(n=num_samples, replace=False, random_state=42)
df_analysis = pd.concat([pos_samples, neg_samples])
print(len(df_analysis[df_analysis['drought_estimated']]), len(df_analysis[~df_analysis['drought_estimated']]))

85 85
